In [ ]:
!pip install -r ../input/requirements/requirements.txt
!pip install -q grape -U
!pip install -q plot_keras_history seedir silence_tensorflow
!pip install -q tsnecuda==3.0.0+cu110 -f https://tsnecuda.isx.ai/tsnecuda_stable.html --no-dependencies
!pip install -q MulticoreTSNE
!pip install -q faiss
!pip install keras_bed_sequence keras_mixed_sequence -U
!pip install extra_keras_metrics
!pip install keras-tuner
!pip install loguru
!pip install barplots
!pip install epigenomic_dataset>=1.1.7

In [6]:
# ===============
# DIRECTORY
# ===============

TUNER_DIR = "../input/tuners/tuners/tuner_hp_" 

MODEL_TYPE_FFNN = "ffnn"
MODEL_TYPE_CNN = "cnn"
MODEL_TYPE_MMNN = "mmnn"

TUNER_DIR_FFNN = TUNER_DIR+MODEL_TYPE_FFNN
TUNER_DIR_CNN = TUNER_DIR+MODEL_TYPE_CNN
TUNER_DIR_MMNN = TUNER_DIR+MODEL_TYPE_MMNN

TUNER_PROJECT_NAME_FFNN = "bioinformatics_project_ffnn_hp"
TUNER_PROJECT_NAME_CNN = "bioinformatics_project_cnn_hp"
TUNER_PROJECT_NAME_MMNN = "bioinformatics_project_mmnn_hp"

MODELS_TYPE = [MODEL_TYPE_FFNN, MODEL_TYPE_CNN, MODEL_TYPE_MMNN]

HP_MAX_EPOCHS_FFNN = 30
HP_MAX_EPOCHS_CNN = 35
HP_MAX_EPOCHS_MMNN = 25

# ===============
# DATA
# ===============

HOLDOUTS_NUM_SPLIT = 10
TEST_SIZE = 0.2

# ===============
# EPIGENOMIC DATA
# ===============

WINDOW_SIZE = 256
CELL_LINE = "H1"
GENOME_CACHE_DIR = "./bio_data/genomes"

# ===============
# FFNN
# ===============

FFNN_NAME_HP = "ffnn_hp"
FFNN_NAME = "BinaryClassificationFFNN"

# ===============
# CNN
# ===============

CNN_NAME_HP = "ffnn_hp"
CNN_NAME = "BinaryClassificationCNN"

# ===============
# MMNN
# ===============

MMNN_NAME_HP = "mmnn_hp"
MMNN_SIMPLE = "MMNN"
MMNN_BOOST = "BoostedMMNN"


In [7]:
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy
from keras_mixed_sequence import MixedSequence, VectorSequence
from keras_bed_sequence import BedSequence
from cache_decorator import Cache
from multiprocessing import cpu_count
from ucsc_genomes_downloader import Genome

from sklearn.impute import KNNImputer
from sklearn.preprocessing import RobustScaler

import pandas as pd
import numpy as np

from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

from typing import Dict, List, Tuple, Optional, Union

from pathlib import Path
from datetime import datetime
import time


def get_cnn_sequence(
        genome: Genome,
        bed: pd.DataFrame,
        y: np.ndarray,
        batch_size: int = 1024
) -> MixedSequence:
    """Returns sequence to train a CNN model on genomic sequences.

    Implementative details
    -------------------------
    This sequence can be used for either binary classification or
    for regresssion, just change the y accordingly.

    Parameters
    -------------------------
    genome: Genome,
        The genome from where to extract the genomic sequence.
    bed: pd.DataFrame,
        The BED file coordinates describing where to extract the sequences.
    y: np.ndarray,
        The values the model should predict.
    batch_size: int = 1024,
        The size of the batches to generate

    Returns
    --------------------------
    MixedSequence object to train a CNN.
    """
    return MixedSequence(
        x={
            "input_sequence_data": BedSequence(
                genome,
                bed,
                batch_size=batch_size,
            )
        },
        y=VectorSequence(
            y,
            batch_size=batch_size
        )
    )


def get_ffnn_sequence(
        X: np.ndarray,
        y: np.ndarray,
        batch_size: int = 1024
) -> MixedSequence:
    """Returns sequence to train a FFNN model on epigenomic data.

    Implementative details
    -------------------------
    This sequence can be used for either binary classification or
    for regresssion, just change the y accordingly.

    Parameters
    -------------------------
    X: np.ndarray,
        The vector from where to extract the epigenomic data.
    y: np.ndarray,
        The values the model should predict.
    batch_size: int = 1024,
        The size of the batches to generate

    Returns
    --------------------------
    MixedSequence object to train a FFNN.
    """
    return MixedSequence(
        x={
            "input_epigenomic_data": VectorSequence(
                X,
                batch_size
            )
        },
        y=VectorSequence(
            y,
            batch_size=batch_size
        )
    )


def get_mmnn_sequence(
        genome: Genome,
        bed: pd.DataFrame,
        X: np.ndarray,
        y: np.ndarray,
        batch_size: int = 1024
) -> MixedSequence:
    """Returns sequence to train a MMNN model on both genomic sequences and epigenomic data.

    Implementative details
    -------------------------
    This sequence can be used for either binary classification or
    for regresssion, just change the y accordingly.

    Parameters
    -------------------------
    genome: Genome,
        The genome from where to extract the genomic sequence.
    bed: pd.DataFrame,
        The BED file coordinates describing where to extract the sequences.
    X: np.ndarray,
        The vector from where to extract the epigenomic data.
    y: np.ndarray,
        The values the model should predict.
    batch_size: int = 1024,
        The size of the batches to generate

    Returns
    --------------------------
    MixedSequence object to train a MMNN.
    """
    return MixedSequence(
        x={
            "input_sequence_data": BedSequence(
                genome,
                bed,
                batch_size=batch_size,
            ),
            "input_epigenomic_data": VectorSequence(
                X,
                batch_size
            )
        },
        y=VectorSequence(
            y,
            batch_size=batch_size
        )
    )


@Cache(
    cache_path=[
        "./boruta/kept_features_{_hash}.json",
        "./boruta/discarded_features_{_hash}.json"
    ],
    args_to_ignore=["X_train", "y_train"]
)
def execute_boruta_feature_selection(
        X_train: pd.DataFrame,
        y_train: np.ndarray,
        holdout_number: int,
        task_name: str,
        max_iter: int = 100
):
    """Returns tuple with list of kept features and list of discared features.
    
    Parameters
    --------------------------
    X_train: pd.DataFrame,
        The data reserved for the input of the training of the Boruta model.
    y_train: np.ndarray,
        The data reserved for the output of the training of the Boruta model.
    holdout_number: int,
        The current holdout number.
    task_name: str,
        The name of the task.
    max_iter: int = 100,
        Number of iterations to run Boruta for.

    Returns
    -------
    kept_features: list(),
        List of indices referring to the features to be maintained.
    discarded_features: list(),
        List of indices referring to the features to be eliminated.
    """

    model = RandomForestClassifier(n_jobs=cpu_count(), class_weight='balanced_subsample', max_depth=5)

    # Create the Boruta model
    boruta_selector = BorutaPy(
        model,  # Defining the model that Boruta should use.
        n_estimators='auto',  # We leave the number of estimators to be decided by Boruta.
        verbose=False,
        alpha=0.05,  # p_value
        # In practice one would run at least 100-200 times,
        # until all tentative features are exausted.
        max_iter=max_iter,
        random_state=42,
    )
    # Fit the Boruta model
    boruta_selector.fit(X_train.values, y_train)

    # Get the kept features and discarded features
    kept_features = list(X_train.columns[boruta_selector.support_])
    discarded_features = list(X_train.columns[~boruta_selector.support_])

    # Filter out the unused featured.
    return kept_features, discarded_features


def to_bed(data: pd.DataFrame) -> pd.DataFrame:
    """Return bed coordinates from given dataset."""
    return data.reset_index()[data.index.names]

def normalize_epigenomic_data(
    train_x: np.ndarray,
    test_x: np.ndarray = None
) -> Tuple[np.ndarray]:
    """Return imputed and normalized epigenomic data.

    We fit the imputation and normalization on the training data and
    apply it to both the training data and the test data.

    Parameters
    -------------------------
    train_x: np.ndarray,
        Training data to use to fit the imputer and scaled.
    test_x: np.ndarray = None,
        Test data to be normalized.

    Returns
    -------------------------
    Tuple with imputed and scaled train and test data.
    """
    # Create the imputer and scaler object
    imputer = KNNImputer()
    scaler = RobustScaler()
    # Fit the imputer object
    imputer.fit(train_x)
    # Impute the train and test data
    imputed_train_x = imputer.transform(train_x)
    if test_x is not None:
        imputed_test_x = imputer.transform(test_x)
    # Fit the scaler object
    scaler.fit(imputed_train_x)
    # Scale the train and test data
    scaled_train_x = scaler.transform(imputed_train_x)
    if test_x is not None:
        scaled_test_x = scaler.transform(imputed_test_x)
    if test_x is not None:
        # Return the normalized data
        return scaled_train_x, scaled_test_x
    return scaled_train_x

In [8]:
from cache_decorator import Cache

from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

from typing import Dict, List, Tuple, Optional, Union

from keras_mixed_sequence import MixedSequence

from pathlib import Path
from datetime import datetime
import time


@Cache(
    cache_path=[
        "./model-histories/model_histories/{cell_line}/{task}/{model_name}/{use_feature_selection}/history_{_hash}.csv.xz",
        "./model-performance/model_performance/{cell_line}/{task}/{model_name}/{use_feature_selection}/performance_{_hash}.csv.xz"
    ],
    args_to_ignore=["model", "training_sequence", "test_sequence"]
)
def train_model(
    model: Model,
    model_name: str,
    task: str,
    cell_line: str,
    training_sequence: MixedSequence,
    test_sequence: MixedSequence,
    holdout_number: int,
    use_feature_selection: bool,
    start_time: time
) -> Tuple[pd.DataFrame, pd.DataFrame]:

    """Returns training history and model evaluations.
    
    Parameters
    ---------------------
    model: Model,
        The model to train.
    model_name: str,
        The model name.
    task: str,
        The name of the task.
    cell_line: str,
        Name of the considered cell line.
    training_sequence: MixedSequence,
        The training sequence.
    test_sequence: MixedSequence,
        The test sequence.
    holdout_number: int,
        The number of the current holdout.
    use_feature_selection: bool,
        Whether the model is trained using features that have
        been selected with Boruta or not.

    Returns
    ----------------------
    Tuple with training history dataframe and model evaluations dataframe.
    """

    history = pd.DataFrame(model.fit(
        train_sequence,
        validation_data=test_sequence,
        epochs=1000,
        verbose=False,
        callbacks=[
            EarlyStopping(
                "loss",
                min_delta=0.001,
                patience=2,
                mode="min"
            )
            #TqdmCallback(verbose=1)
        ]
    ).history)
    
    train_evaluation = dict(zip(model.metrics_names, model.evaluate(train_sequence, verbose=False)))
    test_evaluation = dict(zip(model.metrics_names, model.evaluate(test_sequence, verbose=False)))
    train_evaluation["run_type"] = "train"
    test_evaluation["run_type"] = "test"
    
    for evaluation in (train_evaluation, test_evaluation):
        evaluation["model_name"] = model_name
        evaluation["task"] = task
        evaluation["holdout_number"] = holdout_number 
        evaluation["use_feature_selection"] = use_feature_selection
        evaluation["elapsed_time"] = round(time.time() - start_time, 2)

    evaluations = pd.DataFrame([
        train_evaluation,
        test_evaluation
    ])
    
    return history, evaluations


In [9]:
from ucsc_genomes_downloader import Genome

genome = Genome("hg38", cache_directory=GENOME_CACHE_DIR)

In [10]:
from sklearn.model_selection import StratifiedShuffleSplit, ShuffleSplit

holdouts_generator = StratifiedShuffleSplit(
    n_splits=HOLDOUTS_NUM_SPLIT,
    test_size=TEST_SIZE
)

In [11]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras import optimizers
from extra_keras_metrics import get_complete_binary_metrics
from keras_tuner import HyperModel


class FFNNHyperModel(HyperModel):
    def __init__(self, input_shape):
        self.input_shape = input_shape

    def build(self, hp):
        num_layers = hp.Int(name="num_layers", min_value=2, max_value=6)
        n_neurons0 = hp.Int(name="n_neurons0", min_value=32, max_value=256, step=32)
        learning_rate = hp.Choice(name="learning_rate", values=[1e-2, 1e-4])

        input_epigenomic_data = Input((self.input_shape,), name="input_epigenomic_data")
        last_hidden_ffnn = hidden = input_epigenomic_data

        for layer in range(num_layers):
            if layer == (num_layers - 1):
                name = "last_hidden_ffnn"
            else:
                name = None
            if layer >= 2:
                with hp.conditional_scope("num_layers", [3, 4, 5, 6, 6]):
                    n_neurons1 = hp.Int(
                        name="n_neurons1",
                        min_value=16,
                        max_value=256,
                        step=16
                    )

                    hidden = Dense(
                        n_neurons1,
                        activation="relu",
                        kernel_regularizer=None,
                        name=name
                    )(hidden)
                    last_hidden_ffnn = hidden
            else:
                hidden = Dense(
                    n_neurons0,
                    activation="relu",
                    kernel_regularizer=None,
                    name=name
                )(hidden)
                last_hidden_ffnn = hidden

        output_ffnn = Dense(1, activation="sigmoid")(last_hidden_ffnn)

        model = Model(inputs=input_epigenomic_data, outputs=output_ffnn, name=FFNN_NAME_HP)

        model.compile(
            loss="binary_crossentropy",
            optimizer=optimizers.Nadam(learning_rate=learning_rate),
            metrics=get_complete_binary_metrics()
        )

        return model, input_epigenomic_data, last_hidden_ffnn


In [12]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, ReLU
from tensorflow.keras.layers import Conv1D, MaxPool1D, GlobalAveragePooling1D
from tensorflow.keras import optimizers
from extra_keras_metrics import get_complete_binary_metrics
from keras_tuner import HyperModel


class CNNHyperModel(HyperModel):
    def __init__(self, window_size):
        self.window_size = window_size

    def build(self, hp):
        num_conv_layers = hp.Int(name="num_conv_layers", min_value=2, max_value=8, step=1)
        n_neurons0 = hp.Int(name="n_neurons0", min_value=32, max_value=128, step=32)
        kernel_size0 = hp.Int(name="kernel_size0", min_value=5, max_value=8)
        drop_rate = hp.Float(name="drop_rate", min_value=0.0, max_value=0.5)
        drop_rate_out = hp.Float(name="drop_rate_out", min_value=0.0, max_value=0.5)
        learning_rate = hp.Choice("learning_rate", values=[1e-2, 1e-4])
        n_neurons_last_out = hp.Int(name="n_neurons_last_out", min_value=16, max_value=128, step=16)

        input_sequence_data = Input((self.window_size, 4), name="input_sequence_data")
        hidden = input_sequence_data

        for num_conv_layer in range(num_conv_layers):
            if num_conv_layer >= 2:
                with hp.conditional_scope("num_conv_layers", [3, 4, 5, 6, 7, 8]):
                    n_neurons1 = hp.Int(name="n_neurons1", min_value=16, max_value=128, step=16)
                    kernel_size1 = hp.Int(name="kernel_size1", min_value=2, max_value=10)

                    hidden = Conv1D(n_neurons1, kernel_size=kernel_size1)(hidden)
                    hidden = BatchNormalization()(hidden)
                    hidden = ReLU()(hidden)
                    hidden = Dropout(drop_rate)(hidden)
            else:
                hidden = Conv1D(n_neurons0, kernel_size=kernel_size0)(hidden)
                hidden = BatchNormalization()(hidden)
                hidden = ReLU()(hidden)
                hidden = Dropout(drop_rate)(hidden)

            if num_conv_layer % 2 != 0:
                hidden = MaxPool1D(pool_size=2, padding='same')(hidden)

        hidden = GlobalAveragePooling1D()(hidden)
        hidden = BatchNormalization()(hidden)
        hidden = Dropout(drop_rate_out)(hidden)
        last_hidden_cnn = Dense(n_neurons_last_out, activation="relu", name="last_hidden_cnn")(hidden)
        output_cnn = Dense(1, activation="sigmoid", name="output_cnn")(last_hidden_cnn)

        model = Model(inputs=input_sequence_data, outputs=output_cnn, name=CNN_NAME_HP)

        model.compile(
            loss="binary_crossentropy",
            optimizer=optimizers.Nadam(learning_rate=learning_rate),
            metrics=get_complete_binary_metrics()
        )

        return model, input_sequence_data, last_hidden_cnn


In [13]:
from tensorflow.keras.layers import Dense, Concatenate
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
from extra_keras_metrics import get_complete_binary_metrics
from keras_tuner import HyperModel


class MMNNHyperModel(HyperModel):
    def __init__(self, input_epigenomic_data, input_sequence_data, last_hidden_ffnn, last_hidden_cnn):
        self.input_epigenomic_data = input_epigenomic_data
        self.input_sequence_data = input_sequence_data
        self.last_hidden_ffnn = last_hidden_ffnn
        self.last_hidden_cnn = last_hidden_cnn

    def build(self, hp):
        n_neurons_concat = hp.Int(name="n_neurons_concat", min_value=32, max_value=256, step=32)
        learning_rate = hp.Choice("learning_rate", values=[1e-2, 1e-4])

        concatenation_layer = Concatenate()([self.last_hidden_ffnn, self.last_hidden_cnn])

        last_hidden_mmnn = Dense(n_neurons_concat, activation="relu", name="First_Hidden_Layer")(concatenation_layer)
        output_mmnn = Dense(1, activation="sigmoid", name="Output_Layer")(last_hidden_mmnn)

        model = Model(inputs=[self.input_epigenomic_data, self.input_sequence_data], outputs=output_mmnn, name=MMNN_NAME_HP)
        model.compile(
            optimizer=optimizers.Nadam(learning_rate=learning_rate),
            loss="binary_crossentropy",
            metrics=get_complete_binary_metrics()
        )

        return model


In [14]:
from ucsc_genomes_downloader import Genome

from typing import Tuple

from tensorflow.keras.layers import Layer
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from loguru import logger
from typing import Optional

from keras_tuner import Hyperband
import keras_tuner as kt

import numpy as np
import pandas as pd

from datetime import datetime
import datetime


def hyperparameter_tuning(
        train_X: np.ndarray,
        test_X: np.ndarray,
        train_y: np.ndarray,
        test_y: np.ndarray,
        train_bed: pd.DataFrame,
        test_bed: pd.DataFrame,
        genome: Genome,
        window_size: int,
        holdout_number: int,
        task_name: str,
        model_name: str,
        input_layers: Optional[list] = None,
        hidden_layers: Optional[list] = None
)-> Tuple[Model, Layer, Layer]:
    """Returns tuple with list of kept features and list of discared features.

    Parameters
    --------------------------
    train_X: np.ndarray,
        The vector from where to extract the epigenomic. Used for training phase.
    test_X: np.ndarray,
        The vector from where to extract the epigenomic. Used for test phase.
    train_y: np.ndarray,
        The values the model should predict during the training phase.
    test_y: np.ndarray,
        The values the model should predict during the test phase.
    holdout_number: int,
        The current holdout number.
    task_name: str,
        The name of the task.
    model_name: str,
        The name of the model.

    Returns
    -------
    Return FFNN model resulting from hyperparameter optimization, and dictionary
    that contains best hyperparamters results.
    """

    global hyperparam_results, max_epochs, project_name, directory

    task_name = "AEvsIE" if task_name == "active_enhancers_vs_inactive_enhancers" else "APvsIP"

    if model_name == MODEL_TYPE_FFNN:
        hypermodel = FFNNHyperModel(input_shape=train_X.shape[1])
        max_epochs = HP_MAX_EPOCHS_FFNN
        project_name = TUNER_PROJECT_NAME_FFNN
        directory = TUNER_DIR_FFNN
    elif model_name == MODEL_TYPE_CNN:
        hypermodel = CNNHyperModel(window_size)
        max_epochs = HP_MAX_EPOCHS_CNN
        project_name = TUNER_PROJECT_NAME_CNN
        directory = TUNER_DIR_CNN
    elif model_name == MODEL_TYPE_MMNN:
        if input_layers and hidden_layers:
            #print(f"model_name: {model_name} input_layers: {input_layers} hidden_layers: {hidden_layers}")
            #print(input_layers.get("input_epigenomic_data"))
            hypermodel = MMNNHyperModel(input_layers.get("input_epigenomic_data"),
                                        input_layers.get("input_sequence_data"),
                                        hidden_layers.get("last_hidden_ffnn"),
                                        hidden_layers.get("last_hidden_cnn")
                                        )
        max_epochs = HP_MAX_EPOCHS_MMNN
        project_name = TUNER_PROJECT_NAME_MMNN
        directory = TUNER_DIR_MMNN

    tuners = define_tuners(hypermodel,
                           max_epochs=max_epochs,
                           directory=directory,
                           project_name=project_name)

    for tuner in tuners:
        hyperparam_results = tuner_evaluation(tuner,
                                              train_X,
                                              test_X,
                                              train_y,
                                              test_y,
                                              train_bed,
                                              test_bed,
                                              genome,
                                              task_name,
                                              holdout_number,
                                              model_name)

    return hyperparam_results


def tuner_evaluation(tuner, train_X, test_X, train_y, test_y, train_bed, test_bed, genome, task_name, holdout_number, model_name):
    # Overview of the task
    # tuner.search_space_summary()

    # Performs the hyperparameter tuning
    global train_search_seq, valid_search_seq
    model = None

    #logger.info(f"Start hyperparameter tuning for {model_name}")

    if model_name == MODEL_TYPE_FFNN:
        train_search_seq = get_ffnn_sequence(train_X, train_y)
        valid_search_seq = get_ffnn_sequence(test_X, test_y)
    elif model_name == MODEL_TYPE_CNN:
        train_search_seq = get_cnn_sequence(genome, train_bed, train_y)
        valid_search_seq = get_cnn_sequence(genome, test_bed, test_y)
    elif model_name == MODEL_TYPE_MMNN:
        train_search_seq = get_mmnn_sequence(genome, train_bed, train_X, train_y)
        valid_search_seq = get_mmnn_sequence(genome, test_bed, test_X, test_y)

    tuner.search(train_search_seq,
                 validation_data=valid_search_seq,
                 epochs=200,
                 batch_size=256,
                 callbacks=[EarlyStopping(monitor="val_loss", patience=3, verbose=1)]
                 )

    # Show a summary of the search
    # tuner.results_summary()

    # Retrieve the best hyperparameters.
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    model = tuner.hypermodel.build(best_hps)

    if model_name == MODEL_TYPE_FFNN:
        #print(f"Get Layer From {model_name} Models!")
        input_epigenomic_data = model[1]
        last_hidden_ffnn = model[2]

    if model_name == MODEL_TYPE_CNN:
        #print(f"Get Layer From {model_name} Models!")
        input_sequence_data = model[1]
        last_hidden_cnn = model[2]

    results = {
        "task_name": task_name,
        "holdout_number": holdout_number,
        "learning_rate": best_hps.get("learning_rate"),
        "create_date": datetime.datetime.now().strftime("%d/%m/%Y-%H:%M:%S")
    }

    if model_name == MODEL_TYPE_FFNN:
        results["num_layers"] = best_hps.get("num_layers")
        results["n_neurons0"] = best_hps.get("n_neurons0")
        results["n_neurons1"] = best_hps.get("n_neurons1") if best_hps.get("num_layers") >= 3 else None
        results["input_epigenomic_data"] = input_epigenomic_data
        results["last_hidden_ffnn"] = last_hidden_ffnn

    if model_name == MODEL_TYPE_CNN:
        results["num_layers"] = best_hps.get("num_conv_layers")
        results["n_neurons0"] = best_hps.get("n_neurons0")
        results["kernel_size0"] = best_hps.get("kernel_size0")
        results["drop_rate"] = best_hps.get("drop_rate")
        results["n_neurons1"] = best_hps.get("n_neurons1") if best_hps.get("num_conv_layers") >= 3 else None
        results["kernel_size1"] = best_hps.get("kernel_size1") if best_hps.get("num_conv_layers") >= 3 else None
        results["n_neurons_last_out"] = best_hps.get("n_neurons_last_out")
        results["drop_rate_out"] = best_hps.get("drop_rate_out")
        results["input_sequence_data"] = input_sequence_data
        results["last_hidden_cnn"] = last_hidden_cnn

    if model_name == MODEL_TYPE_MMNN:
        results["n_neurons_concat"] = best_hps.get("n_neurons_concat")

    return {model_name: model, f"{model_name}_parameters": results}


def define_tuners(hypermodel, max_epochs, directory, project_name):
    hyperband_tuner = Hyperband(
        hypermodel,
        max_epochs=max_epochs,
        objective=kt.Objective("val_AUPRC", direction="max"),
        factor=3,
        directory=directory,
        project_name=project_name
    )
    return [hyperband_tuner]


In [15]:
from tqdm.auto import tqdm
from epigenomic_dataset import active_enhancers_vs_inactive_enhancers, active_promoters_vs_inactive_promoters
import warnings
warnings.filterwarnings('ignore')

all_tuner_results = []
all_input_layer = []
all_output_layer = []

for task, threshold in tqdm((
    (active_enhancers_vs_inactive_enhancers, 0),
    (active_promoters_vs_inactive_promoters, 1)
), desc="Tasks"):
    all_results = []
    task_name = task.__name__

    # We get the task data with binarized labels
    X, y = task(
        binarize=True,
        cell_line=CELL_LINE,
        window_size=WINDOW_SIZE,
        min_active_tpm_value=threshold,
        max_inactive_tpm_value=threshold,
        root="./bio_data/epigenomic/"+str(task_name),
        verbose=1
    )
    bed = to_bed(X)

    # Start the main loop, iterating through the holdouts
    for holdout_number, (train_indices, test_indices) in tqdm(
        enumerate(holdouts_generator.split(X, y)),
        total=HOLDOUTS_NUM_SPLIT,
        leave=False,
        desc="Computing Holdouts For {}".format(task_name)
    ):
        # Get the training and test data
        train_bed, test_bed = bed.iloc[train_indices], bed.iloc[test_indices]
        train_X, test_X = X.iloc[train_indices], X.iloc[test_indices]
        train_y, test_y = y.iloc[train_indices], y.iloc[test_indices]

        # Impute and normalize the epigenomic data
        train_X, test_X = normalize_epigenomic_data(train_X, test_X)

        # Flatten the output values
        train_y = train_y.values.flatten()
        test_y = test_y.values.flatten()

        input_layers = {}
        output_layers = {}
        for model in MODELS_TYPE:
            if model == MODEL_TYPE_MMNN:
                check_ffn_param = any("ffnn_parameters" in d for d in all_tuner_results[:2])
                check_cnn_param = any("cnn_parameters" in d for d in all_tuner_results[:2])
                if check_ffn_param and check_cnn_param:
                    input_layers["input_epigenomic_data"] = all_tuner_results[0].get("ffnn_parameters").get("input_epigenomic_data")
                    input_layers["input_sequence_data"] = all_tuner_results[1].get("cnn_parameters").get("input_sequence_data")
                    output_layers["last_hidden_ffnn"] = all_tuner_results[0].get("ffnn_parameters").get("last_hidden_ffnn")
                    output_layers["last_hidden_cnn"] = all_tuner_results[1].get("cnn_parameters").get("last_hidden_cnn")
                    #print(f"check_ffn_param: {check_ffn_param} check_cnn_param: {check_cnn_param} input_layers: {input_layers} output_layers: {output_layers} ")
            tuner_result = hyperparameter_tuning(train_X,
                                                test_X,
                                                train_y,
                                                test_y,
                                                train_bed,
                                                test_bed,
                                                genome,
                                                WINDOW_SIZE,
                                                holdout_number,
                                                task_name, model,
                                                input_layers,
                                                output_layers)
            all_tuner_results.append(tuner_result)

In [16]:
all_tuner_results[:3]

In [17]:
from typing import Tuple
import silence_tensorflow.auto
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Layer
from tensorflow.keras import optimizers
from extra_keras_metrics import get_complete_binary_metrics


def build_binary_classification_ffnn(
    input_shape: int,
    hp_param: dict()
) -> Tuple[Model, Layer, Layer]:
    """Build a custom Feed-Forward Neural Network.

    Parameters
    ----------
    input_shape: int,
        Number of features in the input layer.
    hp_param : dict
        Dictionary with best hyperparameters used for buil net.

    Returns
    -------
    The compiled FFNN.
    """

    num_layers = hp_param.get('num_layers')
    n_neurons0 = hp_param.get('n_neurons0')
    n_neurons1 = hp_param.get('n_neurons1')
    learning_rate = hp_param.get('learning_rate')

    input_epigenomic_data = Input(shape=(input_shape,), name="input_epigenomic_data")
    last_hidden_ffnn = hidden = input_epigenomic_data

    for layer in range(num_layers):
        if layer == (num_layers - 1):
            name = "last_hidden_ffnn"
        else:
            name = None
        if layer >= 2:
            hidden = Dense(
                n_neurons1,
                activation="relu",
                kernel_regularizer=None,
                name=name
            )(hidden)
            last_hidden_ffnn = hidden
        else:
            hidden = Dense(
                n_neurons0,
                activation="relu",
                kernel_regularizer=None,
                name=name
            )(hidden)
            last_hidden_ffnn = hidden

    output_ffnn = Dense(1, activation="sigmoid")(last_hidden_ffnn)

    ffnn = Model(inputs=input_epigenomic_data,
                  outputs=output_ffnn,
                  name=FFNN_NAME)

    ffnn.compile(
        optimizer=optimizers.Nadam(learning_rate=learning_rate),
        loss="binary_crossentropy",
        metrics=get_complete_binary_metrics()
    )

    return ffnn, input_epigenomic_data, last_hidden_ffnn

In [18]:
from typing import Tuple
import silence_tensorflow.auto
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Conv1D, MaxPool1D, GlobalAveragePooling1D
from tensorflow.keras import optimizers
from tensorflow.python.layers.base import Layer
from extra_keras_metrics import get_complete_binary_metrics


def build_binary_classification_cnn(
        window_size: int,
        hp_param: dict()
) -> Tuple[Model, Layer, Layer]:
    """Returns Convolutional Neural Network model for binary classification.

    Parameters
    -----------------------
    window_size: int,
        Size of the input genomic window.

    Returns
    -----------------------
    Triple with model, input layer and output layer.
    """

    num_layers = hp_param.get("num_layers")
    n_neurons0 = hp_param.get("n_neurons0")
    kernel_size0 = hp_param.get("kernel_size0")

    n_neurons1 = hp_param.get("n_neurons1")
    kernel_size1 = hp_param.get("kernel_size1")

    drop_rate = hp_param.get("drop_rate")

    n_neurons_last_out = hp_param.get("n_neurons_last_out")
    drop_rate_out = hp_param.get("drop_rate_out")

    learning_rate = hp_param.get("learning_rate")

    input_sequence_data = Input(shape=(window_size, 4), name="input_sequence_data")
    hidden = Conv1D(n_neurons0, kernel_size=kernel_size0, activation="relu")(input_sequence_data)

    for _ in range(num_layers):
        hidden = Conv1D(
            n_neurons1,
            kernel_size=kernel_size1,
            activation="relu",
        )(hidden)
        hidden = Dropout(rate=drop_rate)(hidden)
        hidden = MaxPool1D(pool_size=2, padding='same')(hidden)

    hidden = GlobalAveragePooling1D()(hidden)
    hidden = BatchNormalization()(hidden)
    hidden = Dropout(rate=drop_rate_out)(hidden)
    last_hidden_cnn = Dense(n_neurons_last_out, activation="relu")(hidden)
    output_cnn = Dense(1, activation="sigmoid")(last_hidden_cnn)

    cnn = Model(
        inputs=input_sequence_data,
        outputs=output_cnn,
        name=CNN_NAME
    )

    cnn.compile(
        optimizer=optimizers.Nadam(learning_rate=learning_rate),
        loss="binary_crossentropy",
        metrics=get_complete_binary_metrics()
    )
    return cnn, input_sequence_data, last_hidden_cnn

In [19]:
from typing import Optional
import silence_tensorflow.auto
from tensorflow.keras.layers import Dense, Concatenate
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
from tensorflow.python.layers.base import Layer

from extra_keras_metrics import get_complete_binary_metrics


def build_binary_classification_mmnn(
        hp_param_mmnn: dict(),
        hp_param_ffnn: Optional[dict] = None,
        hp_param_cnn: Optional[dict] = None,
        input_shape: Optional[int] = None,
        window_size: Optional[int] = None,
        input_epigenomic_data: Optional[Layer] = None,
        input_sequence_data: Optional[Layer] = None,
        last_hidden_ffnn: Optional[Layer] = None,
        last_hidden_cnn: Optional[Layer] = None,
) -> Model:
    """Returns Multi-Modal Neural Network model for binary classification.

    Implementative details
    -----------------------
    If the input shape / window size is not provided and the input layers and
    the feature selection layers are provided, then the network will start
    to train from those layers (which are expected to be pre-trained).
    Conversely, it will create the submodules for the epigenomic and sequence
    data ex-novo.

    Parameters
    -----------------------
    input_shape: Optional[int] = None,
        Number of features in the input layer.
        Either the input shape or the input and output layers of the FFNN
        must be provided.
    window_size: int,
        Size of the input genomic window.
        Either the window size or the input and output layers of the CNN
        must be provided.
    input_epigenomic_data: Optional[Layer] = None,
        Input for the epigenomic data from a FFNN model.
        Either the input shape or the input and output layers of the FFNN
        must be provided.
    input_sequence_data: Optional[Layer] = None,
        Input for the sequence data from a CNN model.
        Either the window size or the input and output layers of the CNN
        must be provided.
    last_hidden_ffnn: Optional[Layer] = None,
        Feature selection layer from a FFNN model.
        Either the input shape or the input and output layers of the FFNN
        must be provided.
    last_hidden_cnn: Optional[Layer] = None,
        Feature selection layer from a CNN model.
        Either the window size or the input and output layers of the CNN
        must be provided.

    Raises
    -----------------------
    ValueError,
        If the input shape is not provided and the input layer and feature selection
        layer of the FFNN are not provided.
    ValueError,
        If the window size is not provided and the input layer and feature selection
        layer of the CNN are not provided.

    Returns
    -----------------------
    Triple with model, input layer and output layer.
    """

    learning_rate = hp_param_mmnn.get("learning_rate")
    n_neurons_concat = hp_param_mmnn.get("n_neurons_concat")

    if input_shape is None and (last_hidden_ffnn is None or input_epigenomic_data is None):
        raise ValueError(
            "Either the input shape or the features selection layer and the input epigenomic "
            "layer must be provided."
        )
    if window_size is None and (last_hidden_cnn is None or input_sequence_data is None):
        raise ValueError(
            "Either the input shape or the features selection layer and the input sequence "
            "layer must be provided."
        )

    if input_shape is not None and hp_param_ffnn is not None:
        _, input_epigenomic_data, last_hidden_ffnn = build_binary_classification_ffnn(input_shape, hp_param_ffnn)

    if window_size is not None and hp_param_cnn is not None:
        _, input_sequence_data, last_hidden_cnn = build_binary_classification_cnn(window_size, hp_param_cnn)

    concatenation_layer = Concatenate()([
        last_hidden_ffnn,
        last_hidden_cnn
    ])

    last_hidden_mmnn = Dense(n_neurons_concat, activation="relu", name="last_hidden_mmnn")(concatenation_layer)
    output_mmnn = Dense(1, activation="sigmoid")(last_hidden_mmnn)

    mmnn = Model(
        inputs=[input_epigenomic_data, input_sequence_data],
        outputs=output_mmnn,
        name=MMNN_BOOST if input_shape is None else MMNN_SIMPLE
    )

    mmnn.compile(
        optimizer=optimizers.Nadam(learning_rate=learning_rate),
        loss="binary_crossentropy",
        metrics=get_complete_binary_metrics()
    )

    return mmnn

In [20]:
from tqdm.auto import tqdm
from epigenomic_dataset import active_enhancers_vs_inactive_enhancers, active_promoters_vs_inactive_promoters
import time
import warnings
warnings.filterwarnings('ignore')

# Create a list to store all the computed performance
all_binary_classification_performance = []

training_histories = {}

# For each task
for task, threshold in tqdm((
    (active_enhancers_vs_inactive_enhancers, 0),
    (active_promoters_vs_inactive_promoters, 1)
), desc="Tasks"):
    start_time = time.time()
    task_name = task.__name__
    # We get the task data with binarized labels
    X, y = task(
        binarize=True,
        cell_line=CELL_LINE,
        window_size=WINDOW_SIZE,
        min_active_tpm_value=threshold,
        max_inactive_tpm_value=threshold,
        root="./bio_data/epigenomic/"+str(task_name),
        verbose=1
    )
    training_histories[task_name] = []

    # Start the main loop, iterating through the holdouts
    for holdout_number, (train_indices, test_indices) in tqdm(
        enumerate(holdouts_generator.split(X, y)),
        total=HOLDOUTS_NUM_SPLIT,
        leave=False,
        desc="Computing Holdouts For {}".format(task_name)
    ):

        for use_feature_selection in tqdm((True, False), desc="Running Feature Selection For {}".format(task_name), leave=False):
            # Get the training and test data
            train_bed, test_bed = bed.iloc[train_indices], bed.iloc[test_indices]
            train_X, test_X = X.iloc[train_indices], X.iloc[test_indices]
            train_y, test_y = y.iloc[train_indices], y.iloc[test_indices]

            # Impute and normalize the epigenomic data
            train_X, test_X = normalize_epigenomic_data(train_X, test_X)

            # Flatten the output values
            train_y = train_y.values.flatten()
            test_y = test_y.values.flatten()

            if use_feature_selection:
                kept_features, discarded_features = execute_boruta_feature_selection(
                    X_train=pd.DataFrame(train_X),
                    y_train=train_y,
                    holdout_number=holdout_number,
                    task_name=task_name,
                    max_iter=20
                )

                if len(kept_features) > 0:
                    train_X = train_X[:,kept_features]
                    test_X = test_X[:,kept_features]

            # Get the number of features of this specific dataset
            number_of_features = train_X.shape[1]
            ffnn_parameters = all_tuner_results[0].get("ffnn_parameters")
            cnn_parameters = all_tuner_results[1].get("cnn_parameters")
            mmnn_parameters = all_tuner_results[2].get("mmnn_parameters")

            ffnn, input_epigenomic_data, last_hidden_ffnn = build_binary_classification_ffnn(input_shape=number_of_features, hp_param=ffnn_parameters)
            
            cnn, input_sequence_data, last_hidden_cnn = build_binary_classification_cnn(window_size=WINDOW_SIZE, hp_param=cnn_parameters)

            mmnn_simple = build_binary_classification_mmnn(hp_param_ffnn=ffnn_parameters,
                                                           hp_param_cnn=cnn_parameters,
                                                           hp_param_mmnn=mmnn_parameters,
                                                           input_shape=number_of_features,
                                                           window_size=WINDOW_SIZE)
            
            mmnn_boosted = build_binary_classification_mmnn(
                hp_param_mmnn=mmnn_parameters,
                input_sequence_data=input_sequence_data,
                input_epigenomic_data=input_epigenomic_data,
                last_hidden_ffnn=last_hidden_ffnn,
                last_hidden_cnn=last_hidden_cnn
            )
            
            for model, train_sequence, test_sequence in tqdm(
                (
                    (ffnn, get_ffnn_sequence(train_X, train_y), get_ffnn_sequence(test_X, test_y)),
                    (cnn, get_cnn_sequence(genome, train_bed, train_y), get_cnn_sequence(genome, test_bed, test_y)),
                    (mmnn_simple, get_mmnn_sequence(genome, train_bed, train_X, train_y), get_mmnn_sequence(genome, test_bed, test_X, test_y)),
                    (mmnn_boosted, get_mmnn_sequence(genome, train_bed, train_X, train_y), get_mmnn_sequence(genome, test_bed, test_X, test_y)),
                ),
                desc="Training models",
                leave=False
            ):

                # We compute the model performance
                history, performance = train_model(
                    model,
                    model.name+"V1",
                    task_name,
                    CELL_LINE,
                    train_sequence,
                    test_sequence,
                    holdout_number,
                    use_feature_selection,
                    start_time
                )
                training_histories[task_name].append(history)
                # We chain the computed performance to the performance list
                all_binary_classification_performance.append(performance)

                start_time = time.time()

# We convert the computed performance list into a DataFrame
all_binary_classification_performance = pd.concat(all_binary_classification_performance)

In [ ]:
!zip -r model-performance.zip ./model-performance

In [ ]:
!zip -r model-histories.zip ./model-histories

In [ ]:
!zip -r boruta.zip ./boruta

In [ ]:
!zip -r barplots.zip ./barplots

In [ ]:
from os import walk
for (dirpath, dirnames, filenames) in walk("./training_histories_pe_vs_pe"):
    print("Directory path: ", dirpath)
    print("Folder name: ", dirnames)
    print("File name: ", filenames)

In [79]:
import os
os.mkdir("./training_histories_ae_vs_ie")

In [80]:
import os
os.mkdir("./training_histories_pe_vs_pe")

In [82]:
for index, x in enumerate(training_histories["active_enhancers_vs_inactive_enhancers"]):
    x.to_csv(f"./training_histories_ae_vs_ie/active_enhancers_vs_inactive_enhancers_{index}.csv")

In [85]:
for index, x in enumerate(training_histories["active_promoters_vs_inactive_promoters"]):
    x.to_csv(f"./training_histories_pe_vs_pe/active_promoters_vs_inactive_promoters{index}.csv")

In [ ]:
!zip -r training_histories_ae_vs_ie.zip ./training_histories_ae_vs_ie

In [ ]:
!zip -r training_histories_pe_vs_pe.zip ./training_histories_pe_vs_pe

In [24]:
import datetime

current_timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
all_binary_classification_performance.to_csv(f"all_binary_classification_performance_{current_timestamp}.csv", index=False)

In [25]:
all_binary_classification_performance

In [26]:
all_binary_classification_performance[all_binary_classification_performance['run_type'] == 'test'].sort_values(by='AUPRC', ascending=False)

In [27]:
# Slightly adapting the dataframe in order to visualiza it better
all_binary_classification_performance["use_feature_selection"] = [
    "Feature Selection" if use_selection else "No Feature Selection"
    for use_selection in all_binary_classification_performance["use_feature_selection"]
]
all_performance = all_binary_classification_performance.drop(columns=["holdout_number"])

In [28]:
from plot_keras_history import plot_history

plot_history(training_histories["active_enhancers_vs_inactive_enhancers"][14], title="active_enhancers_vs_inactive_enhancers", graphs_per_row=6)
plot_history(training_histories["active_promoters_vs_inactive_promoters"][31], title="active_promoters_vs_inactive_promoters", graphs_per_row=6)

In [29]:
from plot_keras_history import plot_history

for region, x in training_histories.items():
  plot_history(training_histories[region], title=region, graphs_per_row=6)

In [33]:
from barplots import barplots

column_to_filter = ["model_name", "loss", "accuracy", "AUROC", "AUPRC", "use_feature_selection", "run_type"]
barplots(
    all_performance[column_to_filter],
    groupby=["model_name", "use_feature_selection", "run_type"],
    orientation="horizontal",
    height=8,
    bar_width=0.2
)

In [34]:
from scipy.stats import wilcoxon

for model in all_binary_classification_performance.model_name.unique():
    model_performance = all_performance[(all_performance.model_name == model) & (all_performance.run_type == "test")]
    performance_with_feature_selection = model_performance[
        all_performance.use_feature_selection == "Feature Selection"
    ]
    performance_without_feature_selection = model_performance[
        all_performance.use_feature_selection == "No Feature Selection"
    ]

    for metric in ("AUPRC", "AUROC", "accuracy"):
        print(
            model,
            metric,
            wilcoxon(performance_with_feature_selection[metric], performance_without_feature_selection[metric])
        )
    print("="*100)

In [ ]:
from scipy.stats import wilcoxon

for model in all_binary_classification_performance.model_name.unique():
    model_performance = all_performance[(all_performance.model_name == model) & (all_performance.run_type == "test")]
    performance_with_feature_selection = model_performance[
        all_performance.use_feature_selection == "Feature Selection"
    ]
    performance_without_feature_selection = model_performance[
        all_performance.use_feature_selection == "No Feature Selection"
    ]

    for metric in ("AUPRC", "AUROC", "accuracy"):
        try:
                _, p_value = wilcoxon(
                    performance_with_feature_selection[metric],
                    performance_without_feature_selection[metric])
                # when p_value is less than the significance level, then the null
                # hypothesis is rejected and the model with greater mean for the
                # current metric is better than the other.
                if p_value < 0.01:
                    if performance_with_feature_selection.mean() > performance_without_feature_selection.mean():
                        print(f"{model} shows better results for {metric} with feature selection.")
                    else:
                        print(f"{model} shows better results for {metric} without feature selection.")
                else:
                    print(f"{model} shows statistically indistinguishiable results for {metric} with and without feature selection.")
        except ValueError:
            # this situation occurs when the matric are exactly the same.
            print(f"{model} has the same values for {metric}")
    print("="*130)